In [1]:
import gym
import numpy as np
env = gym.make('FrozenLake-v1')

In [2]:
# Q Table을 모두 0으로 초기화 : 2차원 (number of state, action space) = (16,4)
# 현재 state에서 어떤 action을 취할 때 얻을 수 있는 reward를 저장하는 리스트
# Q : 주어진 state에서 어떤 action을 취할 것인가에 대한 길잡이
# env.observation_space.n : 환경의 경우의 수
# env.action_space.n : 행동의 경우의 수
Q = np.zeros([env.observation_space.n, env.action_space.n])
print(Q.shape)
# 하이퍼 파라미터 초기화

# 할인율(discount) 정의 => 미래의 보상(reward)을 현재의 보상보다 조금 낮게 계산
dis = 0.99

# 시도 횟수(에피소드)
num_episodes = 2000

# 에피소드마다 총 보상의 합을 저장하는 리스트
rList = []

(16, 4)


In [3]:
# Q 업데이트 : Q값은 어떤 상태에서 어떤 액션을 취할 때 얻어지는 보상 + action으로 변화된 state에서 얻을 수 있는 reward의 최대값을 더한다.
# 즉, 현재의 보상 + 미래에 가능한 보상의 최대값
# 할인율: 미래의 보상에 약간의 패널티를 주는 것
# Q(state, action) = Reward + max(Q(new state))

In [4]:
# 랜덤 방식

frames = []

for i in range(num_episodes):
    state = env.reset()
    rAll = 0
    done = False

    while not done:
        # Action 중에 가장 R(Reward)이 큰 Action을 랜덤으로 고르는 방식
        # env.action_space.n: 4
        # randn(1,4) 1행 4열의 정규분포난수
        action = np.argmax(Q[state, :] + np.random.randn(1, env.action_space.n) / (i + 1))
        # 해당 Action을 했을 때 environment가 변하고, 새로운 state, reward, done 여부를 반환 받음
        new_state, reward, done, _ = env.step(action)

        # Q = R + 할인율 * max(Q)
        Q[state, action] = reward + dis * np.max(Q[new_state, :])
        rAll += reward
        state = new_state

        # 애니메이션을 위하여 정보 기록
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': new_state,
            'action': action,
            'reward': reward
            }
        ) 
    rList.append(rAll)

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

In [10]:
# E-greedy 방식
# 어떠한 확률값 E를 주어 E의 확률로 탐색
# E=0.99라면 99%의 확률로 탐색하고 1%의 확률로 개척(새로운 길을 찾음)

frames = []

for i in range(num_episodes):
    state = env.reset()
    rAll = 0
    done = False
    
    # exploration의 확률
    e = 1./((i/100) +1)

    # Q learning 알고리즘
    while not done:
        # E-Greedy 알고리즘으로 action 고르기
        if np.random.rand(1) < e:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state, :])

        # 해당 Action을 했을 때 environment가 변하고, 새로운 state, reward, done 여부를 반환 받음
        new_state, reward, done, _ = env.step(action)

        # Q = R + Q
        Q[state, action] = reward + dis * np.max(Q[new_state, :])
        rAll += reward
        state = new_state

        # 애니메이션을 위하여 정보 기록
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': new_state,
            'action': action,
            'reward': reward
            }
        ) 
    rList.append(rAll)

In [11]:
print("Success rate : " + str(sum(rList) / num_episodes))
print("Final Q-Table Values")
print(Q)

Success rate : 0.099
Final Q-Table Values
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 5.39113093e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.28384742e-06]
 [0.00000000e+00 0.00000000e+00 6.86175502e-06 6.26833156e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 6.20564825e-06]
 [5.66896670e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.79313747e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.80100000e-01 9.41480149e-01]
 [0.00000000e

In [12]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 251
State: 1
Action: 2
Reward: 0.0


KeyboardInterrupt: 